<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/kaggle_api_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the [Kaggle API](https://github.com/Kaggle/kaggle-api) in Colab

In [16]:
!pip install kaggle
# !pip install google
!conda install -y gdown

/bin/bash: conda: command not found


# Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [17]:
# from google.colab import files
import gdown
!gdown --id 1sD1x-nf2nXNNDFD3zdPKvOKM2wGSWcSP
# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1sD1x-nf2nXNNDFD3zdPKvOKM2wGSWcSP
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 61.9kB/s]


# Using the Kaggle API

For a more complete list of what you can do with the API, visit https://github.com/Kaggle/kaggle-api.

## Listing competitions

In [18]:
# !kaggle competitions list

## Downloading a dataset

In [19]:
# kaggle competitions download -c covid-19-risk-2022

In [20]:
!kaggle competitions download -c covid-19-risk-2022

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
example_submission.ipynb: Skipping, found more recently modified local copy (use --force to force download)
train_small.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
!unzip train.csv.zip

!unzip test.csv.zip

!unzip train_small.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               y
y

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
Archive:  train_small.csv.zip
replace train_small.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_small.csv         


In [22]:
import pandas as pd 

train_small = pd.read_csv('/content/train_small.csv')
# train = pd.read_csv('/content/train.csv')
# test = pd.read_csv('/content/test.csv')

train_small.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,labconfirmed_yn,symptomatic_yn,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2021-09,NY,36.0,BRONX,36005.0,0 - 17 years,Male,NaN,NaN,0.0,NaN,Missing,NaN,1,0,NaN,NaN,NaN,NaN
1,2021-09,CA,6.0,SAN JOAQUIN,6077.0,18 to 49 years,Male,NaN,NaN,NaN,NaN,Missing,NaN,1,NaN,NaN,NaN,NaN,NaN
2,2021-09,MA,25.0,MIDDLESEX,25017.0,0 - 17 years,Female,Missing,Unknown,NaN,NaN,Missing,NaN,1,NaN,NaN,NaN,NaN,NaN
3,2021-09,PA,42.0,ERIE,42049.0,65+ years,Male,White,Non-Hispanic/Latino,0.0,0.0,Missing,1.0,1,1,0.0,0,0.0,NaN
4,2021-09,CA,6.0,KERN,6029.0,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,NaN,1,NaN,NaN,NaN,NaN,NaN


## Uploading a Colab notebook to Kaggle Kernels

Bear with us, as this is a little round-about...

### Downloading a notebook from Colab

To download from Colab, use **File** | **Download .ipynb**

In [23]:
user = "ics435"
repo = "ps1-numpy-seismosmsr"
src_dir = "master"
pyfile = "kaggle_api_in_colab.ipynb"
raw_git = 'https://raw.githubusercontent.com/seismosmsr/machine_learning/main/kaggle_api_in_colab.ipynb'

url = f"{raw_git}"

!wget --no-cache --backups=1 {url} -o submission.ipynb

### Then upload the notebook to your Colab runtime

In [24]:
# uploaded = files.upload()
notebook_path = '/content/kaggle_api_in_colab.ipynb'

In [25]:
# uploaded = files.upload()
# notebook_path = list(uploaded.keys())[0]

In [26]:
!mkdir -p export
!mv $notebook_path export/
!kaggle kernels init -p export

Kernel metadata template written to: export/kernel-metadata.json


In [27]:
import re
import random
your_kaggle_username = 'Aron Boettcher'
notebook_title = 'Test Kernel ' + str(random.randint(1,100))
new_kernel_slug = re.sub(r'[^a-z0-9]+', '-', notebook_title.lower())
notebook_path = 'kaggle_api_in_colab.ipynb'

In [28]:
# Documented here: https://github.com/Kaggle/kaggle-api/wiki/Kernel-Metadata
metadata = '''
{
  "id": "%s/%s",
  "title": "%s",
  "code_file": "%s",
  "language": "python",
  "kernel_type": "notebook",
  "is_private": "true",
  "enable_gpu": "false",
  "enable_internet": "true",
  "dataset_sources": [],
  "competition_sources": [],
  "kernel_sources": []
}
''' % (your_kaggle_username, new_kernel_slug, notebook_title, notebook_path)

In [29]:
!echo '$metadata' > export/kernel-metadata.json
!cat export/kernel-metadata.json


{
  "id": "Aron Boettcher/test-kernel-89",
  "title": "Test Kernel 89",
  "code_file": "kaggle_api_in_colab.ipynb",
  "language": "python",
  "kernel_type": "notebook",
  "is_private": "true",
  "enable_gpu": "false",
  "enable_internet": "true",
  "dataset_sources": [],
  "competition_sources": [],
  "kernel_sources": []
}



In [30]:
!kaggle kernels push -p export

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/aronboettcher/test-kernel-89
